# CityBikes

In [14]:
import pandas as pd
import json

In [15]:
# API Base URL 
base_api = 'http://api.citybik.es/v2/networks'
# Read the json
data = pd.read_json(base_api)
# Store formated data in DataFrame
networks = pd.json_normalize(data['networks'])
# Display a sample of the data
print(networks.head(1))
# Show networks ID & City in Poland
print(networks[['id', 'location.city']][networks['location.country'] == 'PL'].reset_index(drop=True))

                      id                   name  \
0  abu-dhabi-careem-bike  Abu Dhabi Careem BIKE   

                                 href   company  \
0  /v2/networks/abu-dhabi-careem-bike  [Careem]   

                                           gbfs_href  location.latitude  \
0  https://dubai.publicbikesystem.net/customer/gb...            24.4866   

   location.longitude location.city location.country system source ebikes  \
0             54.3728     Abu Dhabi               AE    NaN    NaN    NaN   

  license.name license.url scooters instances  
0          NaN         NaN      NaN       NaN  
                                                id  \
0                                        bikeu-bra   
1             komornicki-system-rowerowy-komorniki   
2                                lrm-lublin-lublin   
3                                       metrorower   
4                      oleski-rower-miejski-olesno   
5   piotrkowski-rower-miejski-piotrkow-trybunalski   
6             

### Send a request to CityBikes for the city of your choice. 

In [16]:
# Bike Network in Torun, Poland
network_id = 'torvelo'
# Get data fron API
data = pd.read_json(base_api + f'/{network_id}?fields=stations')
# Filter the data
stations = data['network']['stations'].copy()
# stations

### Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes)

In [17]:
# Save all stations info in a list
stations_list = []
for i, station in enumerate(stations):
    lat = f"{station['latitude']:.6f}"
    lon = f"{station['longitude']:.6f}"
    free = station['free_bikes']
    bikes = 'bike' if free == 1 else 'bikes'
    # print(station['name'])
    # print(f"{i + 1}. ({lat}, {lon}) : {free} {bikes} available")
    # if lat == '43.6532': print(station['name'])
    stations_list.append(
        {
            'latitude': float(lat),
            'longitude': float(lon),
            'bikes_available': int(free)
        },
    )

### Put your parsed results into a DataFrame.

In [18]:
# Store the list in a DataFrame
stations_df = pd.DataFrame(stations_list)
# Print top 5 station with most bikes available
print(stations_df.sort_values(by='bikes_available', ascending=False).head())
# DataFrame Info
stations_df.info()

     latitude  longitude  bikes_available
39  53.015745  18.607203                9
30  53.023882  18.617910                9
20  53.020554  18.570606                9
36  53.021252  18.613106                8
29  53.031525  18.610275                8
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   latitude         51 non-null     float64
 1   longitude        51 non-null     float64
 2   bikes_available  51 non-null     int64  
dtypes: float64(2), int64(1)
memory usage: 1.3 KB


### Save data into CSV file

In [19]:
# Saving data into CSV file
stations_df.to_csv('../data/city_bikes.csv', index=False)